In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import VarianceThreshold

In [ ]:
%pylab inline

In [ ]:
tr = pd.read_csv('input/train.csv')
te = pd.read_csv('input/test.csv')
d = pd.concat([tr, te])
tr = None
te = None

In [ ]:
dl = d.target
del d['target']

In [ ]:
TOP_N = 10

def proc_obj_few(c):
    c = c.fillna('MISSING_VALUE')
    return pd.get_dummies(c, prefix=c.name)

def top_value_or_other_obj(v, top_values):
    if v in top_values:
        return v
    return 'OTHER_VALUE'

def proc_obj_many_conc(c):
    c = c.fillna('MISSING_VALUE')
    top_values = c.value_counts().head(TOP_N).index.tolist()
    c = c.apply(lambda v: top_value_or_other_obj(v, top_values))
    return pd.get_dummies(c, prefix=c.name)

def proc_num_few(c):
    c = c.fillna(-73)
    return DataFrame(c)

def top_value_or_other_num(v, top_values):
    if v in top_values:
        return v
    return -74

def proc_num_many_conc(c):
    c = c.fillna(-73)
    top_values = c.value_counts().head(TOP_N).index.tolist()
    c2 = c.apply(lambda v: top_value_or_other_num(v, top_values))
    ret = DataFrame(c2)
    ret[c.name + '_orig'] = c
    ret[c.name + '_cat'] = pd.cut(c, 3, labels=False)
    return ret

def proc_num_many_not_conc(c):
    c = c.fillna(-73)
    ret = DataFrame(c)
    ret[c.name + '_cat'] = pd.cut(c, 3, labels=False)
    return ret

In [ ]:
def proc_col(c):
    n = len(c)
    if c.dtype == 'object':
        c.replace([np.inf, -np.inf], 'INF_VALUE')
        if len(c.value_counts()) <= TOP_N:
            return proc_obj_few(c)
        else:
            if c.value_counts().head(TOP_N).sum() >= (0.8 * n):
                return proc_obj_many_conc(c)
            else:
                return None
    else:
        c.replace([np.inf, -np.inf], np.nan)
        if len(c.value_counts()) <= TOP_N:
            return proc_num_few(c)
        else:
            if c.value_counts().head(TOP_N).sum() >= (0.8 * n):
                return proc_num_many_conc(c)
            else:
                return proc_num_many_not_conc(c)

In [ ]:
e = DataFrame(index=d.index)
for c in d.columns:
    f = proc_col(d[c])
    del d[c]
    if f is not None:
        for x in f.columns:
            try:
                e[x] = f[x]
            except:
                pass
    print c

In [ ]:
d = None

In [ ]:
e['target'] = dl

In [ ]:
e.to_csv('tmp/engineered_features_02.csv')

In [ ]:
e.info()